In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

StatementMeta(, 204ec58f-2985-40a0-bf16-f0b291e34b18, 3, Finished, Available, Finished)

In [ ]:
# Path to wind_power table in Silver Lakehouse
silver_table_path = "abfss://wind_turbine_energy@onelake.dfs.fabric.microsoft.com/LH_Silver_WindEnergy.Lakehouse/Tables/dbo/wind_power"

# Load the wind_power table into a DataFrame
df = spark.read.format("delta").load(silver_table_path)

StatementMeta(, 204ec58f-2985-40a0-bf16-f0b291e34b18, 4, Finished, Available, Finished)

In [ ]:
# Create the Date Dimension Table
date_dim = df.select("date", "day", "month", "quarter", "year").distinct() \
                .withColumnRenamed("date", "date_id")

# Create the Time Dimension Table
time_dim = df.select("time", "hour_of_day", "minute_of_hour", "second_of_minute", "time_period").distinct() \
                .withColumnRenamed("time", "time_id")

# Create the Turbine Dimension Table
turbine_dim = df.select("turbine_name", "capacity", "location_name", "latitude", "longitude", "region").distinct() \
                .withColumn("turbine_id", row_number().over(Window.orderBy("turbine_name", "capacity", "location_name", "latitude", "longitude", "region")))

# Create the Operational Status Dimension Table
operational_status_dim = df.select("status", "responsible_department").distinct() \
                .withColumn("status_id", row_number().over(Window.orderBy("status", "responsible_department")))

StatementMeta(, 204ec58f-2985-40a0-bf16-f0b291e34b18, 5, Finished, Available, Finished)

In [ ]:
# Join the dimension tables to the original DataFrame
df = df.join(turbine_dim, ["turbine_name", "capacity", "location_name", "latitude", "longitude", "region"], "left") \
        .join(operational_status_dim, ["status", "responsible_department"], "left")

StatementMeta(, 204ec58f-2985-40a0-bf16-f0b291e34b18, 6, Finished, Available, Finished)

In [ ]:
# Create the Fact table
fact_table = df.select("production_id", "date", "time", "turbine_id", "status_id", "wind_speed", "wind_direction", "energy_produced") \
                .withColumnRenamed("date", "date_id") \
                .withColumnRenamed("time", "time_id")

StatementMeta(, 204ec58f-2985-40a0-bf16-f0b291e34b18, 7, Finished, Available, Finished)

In [ ]:
# Paths to the Gold tables
gold_date_dim_path = "abfss://wind_turbine_energy@onelake.dfs.fabric.microsoft.com/LH_Gold_WindEnergy.Lakehouse/Tables/dbo/dim_date"
gold_time_dim_path = "abfss://wind_turbine_energy@onelake.dfs.fabric.microsoft.com/LH_Gold_WindEnergy.Lakehouse/Tables/dbo/dim_time"
gold_turbine_dim_path = "abfss://wind_turbine_energy@onelake.dfs.fabric.microsoft.com/LH_Gold_WindEnergy.Lakehouse/Tables/dbo/dim_turbine"
gold_operational_status_dim_path = "abfss://wind_turbine_energy@onelake.dfs.fabric.microsoft.com/LH_Gold_WindEnergy.Lakehouse/Tables/dbo/dim_operational_status"
gold_fact_table_path = "abfss://wind_turbine_energy@onelake.dfs.fabric.microsoft.com/LH_Gold_WindEnergy.Lakehouse/Tables/dbo/fact_wind_power"

StatementMeta(, 204ec58f-2985-40a0-bf16-f0b291e34b18, 8, Finished, Available, Finished)

In [ ]:
# Save the tables in the Gold Lakehouse
date_dim.write.format("delta").mode("overwrite").save(gold_date_dim_path)
time_dim.write.format("delta").mode("overwrite").save(gold_time_dim_path)
turbine_dim.write.format("delta").mode("overwrite").save(gold_turbine_dim_path)
operational_status_dim.write.format("delta").mode("overwrite").save(gold_operational_status_dim_path)
fact_table.write.format("delta").mode("overwrite").save(gold_fact_table_path)

StatementMeta(, 204ec58f-2985-40a0-bf16-f0b291e34b18, 9, Finished, Available, Finished)